<a href="https://colab.research.google.com/github/pinilDissanayaka/Election-Insight-App/blob/main/Notebook_manifesto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [12]:
import os
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq.chat_models import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import PyPDFLoader
from langchain_pinecone import PineconeVectorStore
from pydantic import BaseModel, Field
from typing import List, TypedDict
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [8]:
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY']=userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'

In [10]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings=HuggingFaceEmbeddings(model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [54]:
def load(path):
  print("=" * 60)
  manifesto_data=PyPDFLoader(path).load()
  print(f"Loaded documents {len(manifesto_data)}")
  print("=" * 60)
  return manifesto_data

def split(docs):
  doc_splits = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=390,
    length_function=len,
    is_separator_regex=False,
  ).split_documents(docs)

  return doc_splits

def store(doc_splits, index_name='manifesto'):
  pinecone_vectore_store=PineconeVectorStore.from_documents(documents=doc_splits, embedding=embeddings, index_name="manifesto")
  return pinecone_vectore_store

In [53]:
manifestos_paths=[
    '/content/NAMAL_VISION 2025-2035 Policy_Targets_and_Innovation_Strategies (En) .pdf',
    '/content/NPP Presidential Election Manifesto - 2024.pdf',
    '/content/Sajith_manifesto_english.pdf'
]


In [49]:
def createIndex(indexName:str, dimension:int):
  try:
    pinecone=Pinecone()
    if indexName not in pinecone.list_indexes().names():
      pinecone.create_index(
          name=indexName,
          dimension=dimension,
          metric="cosine",
          spec=ServerlessSpec(cloud='aws',
                            region='us-east-1')
      )
      print(f"Created {indexName}")
    else:
      print(f"{indexName} already exists")
    print(pinecone.describe_index(indexName))
    return pinecone.Index(indexName)
  except Exception as e:
    print(e.args)

In [50]:
dimention=768

In [51]:
index=createIndex(indexName="manifesto", dimension=dimention)

Created manifesto
{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'manifesto-4myrn7y.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'manifesto',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [ ]:
vector_store=[store(split(load(path))) for path in manifestos_paths]

In [ ]:
pinecone_vectore_store=PineconeVectorStore(embedding=embeddings, index_name="constitution")
retriever=pinecone_vectore_store.as_retriever()

In [ ]:
llm=ChatGroq(model="llama-3.1-70b-versatile")

In [ ]:
class GraphState(TypedDict):
  question:List
  retrieved_docs:List
  genaration:List

In [ ]:
def retrieve(state:GraphState):
  question=state['question'][-1]
  retrieved_docs=retriever.invoke(question)
  return {'retrieved_docs':retrieved_docs, 'question': question}

In [ ]:
def genarate(state:GraphState):
  question=state['question'][-1]
  retrieved_docs=state['retrieved_docs']
  generation=llm.invoke(question=question, context=retrieved_docs)

In [ ]:
class Doc_Grade(BaseModel):
  """Binary score for relevance check on retrieved documents."""
  binary_score:str=Field(description="binary grade for the documents")




def grader(state:GraphState):
  grading_prompt_template="""You are a grader assessing relevance of a retrieved document to a user question. \n
                It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
                If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
                Give a binary_score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

  grading_prompt=ChatPromptTemplate.from_messages([
      ("system", grading_prompt_template),
      ("human", "Retrieved document: \n\n {context} \n\n User question: {question}"),
  ])

  grader_structured_llm=llm.with_structured_output(Doc_Grade)

  grager_chain= grading_prompt | grader_structured_llm | StrOutputParser

In [37]:
type(manifesto_docs[1][0])

langchain_core.documents.base.Document

49

In [41]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
